In [1]:
from tensorflow.python.eager import backprop
from tensorflow.python.keras.engine import data_adapter
from tensorflow.python.keras.mixed_precision.experimental import loss_scale_optimizer as lso
from tensorflow.python.distribute import parameter_server_strategy
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
mnist_train = pd.read_csv('Data/digit-recognizer/train.csv').astype(np.float32)
mnist_test = pd.read_csv('Data/digit-recognizer/test.csv').astype(np.float32)

In [3]:
y_train = mnist_train[['label']][:30000]
x_train = mnist_train[['pixel' + str(idx) for idx in range(784)]][:30000]

y_dev = mnist_train[['label']][30000:42000]
x_dev = mnist_train[['pixel' + str(idx) for idx in range(784)]][30000:42000]

x_test = mnist_test[['pixel' + str(idx) for idx in range(784)]]


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_dev = scaler.transform(x_dev)
x_test = scaler.transform(x_test)

print(x_train.shape, y_train.shape, x_dev.shape, y_dev.shape, x_test.shape)

(30000, 784) (30000, 1) (12000, 784) (12000, 1) (28000, 784)


In [5]:
dropout_rate = 0.0
cl_coeff = 0.1
learning_rate = 1e-4
variance_epsilon = 1e-3
bunch_len = 30000

In [8]:
'''
@tf.function
def update_centers(features, labels, alpha, centers):
    labels = tf.reshape(labels, [-1])
    centers_batch = tf.gather(centers, labels)
    
    diff = centers_batch - features

    unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)
    appear_times = tf.gather(unique_count, unique_idx)
    appear_times = tf.reshape(appear_times, [-1, 1])

    diff = diff / tf.cast((1 + appear_times), tf.float32)
    diff = alpha * diff

    centers = tf.compat.v1.scatter_sub(centers,labels, diff)
    
    return centers



@tf.function
def get_center_loss(features, labels, centers):
    len_features = features.get_shape()[1]
    labels = tf.reshape(labels, [-1])
    centers_batch = tf.gather(centers, labels)
    center_loss = tf.reduce_sum(input_tensor=(features - centers_batch) ** 2, axis=[1])
    return center_loss



y_predict = tf.math.argmax(y_conv, axis = 1)

center_loss = get_center_loss(h_fc1, y_predict, centers)

softmax_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.stop_gradient(ys_one_hot), logits=y_conv)

loss = tf.reduce_mean(input_tensor=softmax_loss ) #+ cl_coeff * center_loss
    

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

var1 = tf.Variable(1.0)
var2 = tf.Variable(1.0)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(var1)
    tape.watch(var2)
    int_val = var1**2
    tmp_loss = int_val * var2

#opt.minimize(tmp_loss, [var1])


grads = tape.gradient(tmp_loss, [var1,var2])
print(grads)
opt.apply_gradients(zip(grads,[var1,var2]))
print(var1,var2)
'''

'\n@tf.function\ndef update_centers(features, labels, alpha, centers):\n    labels = tf.reshape(labels, [-1])\n    centers_batch = tf.gather(centers, labels)\n    \n    diff = centers_batch - features\n\n    unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)\n    appear_times = tf.gather(unique_count, unique_idx)\n    appear_times = tf.reshape(appear_times, [-1, 1])\n\n    diff = diff / tf.cast((1 + appear_times), tf.float32)\n    diff = alpha * diff\n\n    centers = tf.compat.v1.scatter_sub(centers,labels, diff)\n    \n    return centers\n\n\n\n@tf.function\ndef get_center_loss(features, labels, centers):\n    len_features = features.get_shape()[1]\n    labels = tf.reshape(labels, [-1])\n    centers_batch = tf.gather(centers, labels)\n    center_loss = tf.reduce_sum(input_tensor=(features - centers_batch) ** 2, axis=[1])\n    return center_loss\n\n\n\ny_predict = tf.math.argmax(y_conv, axis = 1)\n\ncenter_loss = get_center_loss(h_fc1, y_predict, centers)\n\nsoftmax

In [6]:
ys = y_train[:bunch_len].astype(int).values.reshape(-1)
xs = x_train[:bunch_len]
ys_one_hot = tf.one_hot(ys, 10)


In [21]:

def _minimize(strategy, tape, optimizer, loss, trainable_variables):
    """Minimizes loss for one step by updating `trainable_variables`.
    This is roughly equivalent to
    ```python
    gradients = tape.gradient(loss, trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, trainable_variables))
    ```
    However, this function also applies gradient clipping and loss scaling if the
    optimizer is a LossScaleOptimizer.
    Args:
    strategy: `tf.distribute.Strategy`.
    tape: A gradient tape. The loss must have been computed under this tape.
    optimizer: The optimizer used to minimize the loss.
    loss: The loss tensor.
    trainable_variables: The variables that will be updated in order to minimize
    the loss.
    """

    with tape:
        if isinstance(optimizer, lso.LossScaleOptimizer):
            loss = optimizer.get_scaled_loss(loss)

    gradients = tape.gradient(loss, trainable_variables)

    # Whether to aggregate gradients outside of optimizer. This requires support
    # of the optimizer and doesn't work with ParameterServerStrategy and
    # CentralStroageStrategy.
    aggregate_grads_outside_optimizer = (
        optimizer._HAS_AGGREGATE_GRAD and  # pylint: disable=protected-access
        not isinstance(strategy.extended,
                       parameter_server_strategy.ParameterServerStrategyExtended))

    if aggregate_grads_outside_optimizer:
        # We aggregate gradients before unscaling them, in case a subclass of
        # LossScaleOptimizer all-reduces in fp16. All-reducing in fp16 can only be
        # done on scaled gradients, not unscaled gradients, for numeric stability.
        gradients = optimizer._aggregate_gradients(zip(gradients,  # pylint: disable=protected-access
                                                       trainable_variables))
    if isinstance(optimizer, lso.LossScaleOptimizer):
        gradients = optimizer.get_unscaled_gradients(gradients)
    gradients = optimizer._clip_gradients(gradients)  # pylint: disable=protected-access
    if trainable_variables:
        if aggregate_grads_outside_optimizer:
            optimizer.apply_gradients(
                zip(gradients, trainable_variables),
                experimental_aggregate_gradients=False)
        else:
            optimizer.apply_gradients(zip(gradients, trainable_variables))



class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        #Initialize model variables
        self.W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev=0.1), name="W_conv1")
        self.b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]), name="b_conv1")   
        self.W_conv2 = tf.Variable(tf.random.truncated_normal([5, 5, 32, 64], stddev=0.1), name="W_conv2")
        self.b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]), name="b_conv2")  
        self.W_fc1 = tf.Variable(tf.random.truncated_normal([7 * 7 * 64, 1024], stddev=0.1), name="W_fc1")
        self.b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]), name="b_fc1") 
        self.W_fc2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1), name="W_fc2")
        self.b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]), name="b_fc2")
        #Make centers tf.constants - because we update them manually
        #self.centers = tf.Variable(tf.zeros([10,1024], dtype=tf.float32), name="centers")
        self.centers = tf.constant(tf.zeros([10,1024], dtype=tf.float32), name="centers")

    def get_features(self, x):
        x_reshaped = tf.reshape(x, [-1,28,28,1])
        h_conv1 = tf.nn.relu(tf.nn.conv2d(input=x_reshaped, filters=self.W_conv1, strides=[1, 1, 1, 1], padding='SAME') + self.b_conv1) 
        h_pool1 = tf.nn.max_pool2d(input=h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        h_conv2 = tf.nn.relu(tf.nn.conv2d(input=h_pool1, filters=self.W_conv2, strides=[1, 1, 1, 1], padding='SAME') + self.b_conv2) 
        h_pool2 = tf.nn.max_pool2d(input=h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1_0 = tf.matmul(h_pool2_flat, self.W_fc1)
        mean, var = tf.nn.moments(x=h_fc1_0, axes=[0])
        h_fc1_1 = tf.nn.batch_normalization(h_fc1_0, mean, var, self.b_fc1, 1, variance_epsilon)
        return h_fc1_1
        
    def __call__(self, x, training=False):
        h_fc1 = self.get_features(x)
        h_fc1_drop = tf.cond(tf.equal(training, tf.constant(True)), lambda: tf.nn.dropout(h_fc1, dropout_rate), lambda: h_fc1) 
        y_out = tf.matmul(h_fc1_drop, self.W_fc2) + self.b_fc2
        #In case of library losses use softmax output
        return y_out, h_fc1   #tf.nn.softmax(y_out)

    def get_center_loss(self, features, labels):
        labels = tf.reshape(labels, [-1])
        centers_batch = tf.gather(tf.stop_gradient(self.centers), labels)
        center_loss = tf.reduce_sum(input_tensor=(features - centers_batch) ** 2, axis=[1])
        return center_loss

    def centers_update(self, features, labels, alpha):
        labels = tf.reshape(labels, [-1])
        centers_batch = tf.gather(self.centers, labels)
        diff = centers_batch - features

        unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)
        appear_times = tf.gather(unique_count, unique_idx)
        appear_times = tf.reshape(appear_times, [-1, 1])

        diff = diff / tf.cast((1 + appear_times), tf.float32)
        diff = alpha * diff
        
        label_reshaped = tf.reshape(labels,[-1,1])
        
        self.centers = tf.tensor_scatter_nd_sub(self.centers, label_reshaped, diff)
        
    def get_losses(self, x, y, training=False):
        y_pred, h_features = self(x, training=training)
        
        return self._get_losses(y, y_pred, h_features)

    def _get_losses(self, y, y_pred, h_features):
        y_one_hot = tf.one_hot(y, 10)
        center_loss = self.get_center_loss(h_features, y)
        softmax_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.stop_gradient(y_one_hot), logits=y_pred)
        loss = tf.reduce_mean(input_tensor=softmax_loss + 0.1 * center_loss)    #
        return loss, softmax_loss, center_loss
    
    def get_accuracy(self, x, y):
        y_pred, _ = self(x, training=False)
        result = tf.argmax(input=y_pred,axis=1)
        ground_truth = tf.reshape(y, [-1])
        correct_prediction = tf.equal(result, ground_truth)
        accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
        return accuracy
    
    def get_centers(self):
        return self.centers
    
    def train_step(self, data):
        data = data_adapter.expand_1d(data)
        x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)

        with backprop.GradientTape() as tape:
            y_pred, h_features = self(x, training=True)
            loss, softmax_loss, center_loss = self._get_losses(y, y_pred, h_features) 
            #Default loss cant be used...
            #loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
            
        #For custom training steps, users can just write:
        #trainable_variables = self.trainable_variables
        #gradients = tape.gradient(loss, trainable_variables)
        #self.optimizer.apply_gradients(zip(gradients, trainable_variables))
        #default optimizer cant be used, or not???
        _minimize(self.distribute_strategy, tape, self.optimizer, loss,
                  self.trainable_variables)
        
        #update center loss    
        self.centers_update(h_features, y, 0.5) 
        
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
        
        #trace output
        tf.print({m.name: m.result() for m in self.metrics})
        
        return {m.name: m.result() for m in self.metrics}

    
#Not used here, because cant implement center loss
class Softmax_and_center_loss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        softmax_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.stop_gradient(y_true), logits=y_pred)
        loss = tf.reduce_mean(input_tensor=softmax_loss ) 
        return loss


In [385]:
#old code, couldn't update 

model = Model()
model_loss = Softmax_and_center_loss()
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(opt, loss=model_loss, metrics=["accuracy"])
model.fit(xs,ys, batch_size=1000, epochs = 1)

'model = Model()\nmodel_loss = Softmax_and_center_loss()\nopt = tf.keras.optimizers.Adam(learning_rate=learning_rate)\n\nmodel.compile(opt, loss=model_loss, metrics=["accuracy"])\nmodel.fit(xs,ys, batch_size=1000, epochs = 1)'

In [22]:
model = Model()
model_loss = Softmax_and_center_loss()
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(opt, loss=model_loss, metrics=["accuracy"])

dataset = tf.data.Dataset.from_tensor_slices((xs, ys))
dataset_batches = dataset.batch(1000)

for i in range(3):
    loss, softmax_loss, center_loss = model.get_losses(xs, ys)
    tf.print('Accuracy: ', model.get_accuracy(xs, ys))
    tf.print('Full loss: ', loss)
    tf.print('Softmax loss: ', softmax_loss)
    tf.print('Center loss: ', tf.reduce_mean(center_loss))
    tf.print('Centers: ', model.get_centers())
    for batch in dataset_batches:    
        model.train_step(batch)


Accuracy:  0.1349
Full loss:  107.197105
Softmax loss:  [5.52084494 0.309246838 5.80147219 ... 4.01978683 3.02447319 2.15370369]
Center loss:  1034.14978
Centers:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'accuracy': 0.135}
{'accuracy': 0.161}
{'accuracy': 0.191666663}
{'accuracy': 0.226}
{'accuracy': 0.2632}
{'accuracy': 0.292333335}
{'accuracy': 0.321714282}
{'accuracy': 0.3455}
{'accuracy': 0.364888877}
{'accuracy': 0.3854}
{'accuracy': 0.404}
{'accuracy': 0.419416666}
{'accuracy': 0.43584615}
{'accuracy': 0.45071429}
{'accuracy': 0.463466674}
{'accuracy': 0.478125}
{'accuracy': 0.491470575}
{'accuracy': 0.50372225}
{'accuracy': 0.516473711}
{'accuracy': 0.52865}
{'accuracy': 0.53895241}
{'accuracy': 0.548454523}
{'accuracy': 0.558695674}
{'accuracy': 0.566583335}
{'accuracy': 0.5752}
{'accuracy': 0.582346141}
{'accuracy': 0.590222239}
{'accuracy': 0.596892834}
{'accuracy': 0.603586197}
{'accuracy': 0.60

In [357]:
dataset

TypeError: 'TensorSliceDataset' object does not support indexing

(<tf.Tensor: shape=(1000, 784), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(1000,), dtype=int64, numpy=
array([1, 0, 1, 4, 0, 0, 7, 3, 5, 3, 8, 9, 1, 3, 3, 1, 2, 0, 7, 5, 8, 6,
       2, 0, 2, 3, 6, 9, 9, 7, 8, 9, 4, 9, 2, 1, 3, 1, 1, 4, 9, 1, 4, 4,
       2, 6, 3, 7, 7, 4, 7, 5, 1, 9, 0, 2, 2, 3, 9, 1, 1, 1, 5, 0, 6, 3,
       4, 8, 1, 0, 3, 9, 6, 2, 6, 4, 7, 1, 4, 1, 5, 4, 8, 9, 2, 9, 9, 8,
       9, 6, 3, 6, 4, 6, 2, 9, 1, 2, 0, 5, 9, 2, 7, 7, 2, 8, 8, 5, 0, 6,
       0, 0, 2, 9, 0, 4, 7, 7, 1, 5, 7, 9, 4, 6, 1, 5, 7, 6, 5, 0, 4, 8,
       7, 6, 1, 8, 7, 3, 7, 3, 1, 0, 3, 4, 5, 4, 0, 5, 4, 0, 3, 5, 1, 0,
       8, 3, 7, 0, 9, 6, 6, 9, 5, 4, 6, 9, 3, 5, 4, 2, 4, 8, 7, 7, 5, 8,
       8, 8, 2, 6, 9, 3, 1, 0, 4, 1, 5, 9, 0, 6, 2, 1, 3,

In [309]:
tensor = tf.zeros([3, 1024])
print(tensor)
indices = tf.reshape(tf.constant(tf.ones([5,],dtype=tf.int32)), [-1,1])
print(indices)
updates = tf.constant(tf.ones([5, 1024],dtype=tf.float32))
print(updates)
updated = tf.tensor_scatter_nd_sub(tensor, indices, updates)

print(updated)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(3, 1024), dtype=float32)
tf.Tensor(
[[1]
 [1]
 [1]
 [1]
 [1]], shape=(5, 1), dtype=int32)
tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(5, 1024), dtype=float32)
tf.Tensor(
[[ 0.  0.  0. ...  0.  0.  0.]
 [-5. -5. -5. ... -5. -5. -5.]
 [ 0.  0.  0. ...  0.  0.  0.]], shape=(3, 1024), dtype=float32)


In [327]:
updates = tf.constant(tf.ones([5, 1024],dtype=tf.float32))
updates = updates + updates
updates

<tf.Tensor: shape=(5, 1024), dtype=float32, numpy=
array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)>

In [170]:
model.centers

<tf.Tensor: shape=(10, 1024), dtype=float32, numpy=
array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)>

In [147]:
#xs[0].shape
model.watched_variables()


AttributeError: 'Model' object has no attribute 'watched_variables'

In [144]:
model.predict(xs[:1]) 

array([[4.4806485, 4.4763007, 4.4685955, 4.484938 , 4.537973 , 4.48512  ,
        4.5087147, 4.507668 , 4.504064 , 4.4728446]], dtype=float32)

In [121]:

var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
opt.minimize(loss, [var1]).numpy()


<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=9.99995>

In [227]:
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
tensor = tf.ones([8], dtype=tf.int32)


<tf.Tensor: shape=(71,), dtype=int32, numpy=
array([ 0, 11,  0, 10,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 12], dtype=int32)>

In [35]:
print(ys)
print(ys_one_hot)


tf.Tensor([1 0 1], shape=(3,), dtype=int32)
tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(3, 10), dtype=float32)
